In [1]:
binFile = '../../experiments/problems/bin1data/N1C1W1_A.BPP'
minBins = 26

In [2]:
import pandas as pd
import numpy as np

problemSet = pd.read_csv(binFile, header=None).values.tolist()

PROBLEM_SIZE = problemSet.pop(0)[0]
BIN_CAPACITY = problemSet.pop(0)[0]
POPULATION_SIZE = 50
TOURNAMENT_SIZE = 4
GENERATIONS = 500
SAMPLES = 1
SAMPLE_RATE = 50

MUTATION_RATE = 0.3
CROSSOVER_RATE = 1

K = 4

items = pd.DataFrame(problemSet)
items.head()
items = np.array(items[0])
items

array([99, 99, 96, 96, 92, 92, 91, 88, 87, 86, 85, 76, 74, 72, 69, 67, 67,
       62, 61, 56, 52, 51, 49, 46, 44, 42, 40, 40, 33, 33, 30, 30, 29, 28,
       28, 27, 25, 24, 23, 22, 21, 20, 17, 14, 13, 11, 10,  7,  7,  3])

In [3]:
organisedChromosome = np.arange(items.size)

In [4]:
assert PROBLEM_SIZE == len(items)

In [5]:
def getMappedFitness(chromosome):
    mappedChromosome = items[chromosome]
    spaces = np.zeros(len(mappedChromosome), dtype=int)
    result = np.cumsum(mappedChromosome) - BIN_CAPACITY
    binsRequired = 0
    spacesLeftOpen = []
    while True:
        binsRequired += 1
        max_accumulate = np.maximum.accumulate(np.flipud(result <= 0))
        index_of_new_bin = PROBLEM_SIZE - next((idx for idx, val in np.ndenumerate(max_accumulate) if val == True))[0] - 1
        space_left_open = np.abs(result[index_of_new_bin])
        spaces[index_of_new_bin] = space_left_open
        result += space_left_open
        spacesLeftOpen.append(space_left_open)
        if np.max(result) <= 0:
            break
        result -= BIN_CAPACITY
    return [binsRequired, spacesLeftOpen, BIN_CAPACITY]

In [6]:
def toStringMappedFitness(chromosome):
    result = np.cumsum(problemSet[chromosome]) - BIN_CAPACITY
    output = ''
    while True:
        max_accumulate = np.maximum.accumulate(np.flipud(result <= 0))
        index_of_new_bin = PROBLEM_SIZE - next((idx for idx, val in np.ndenumerate(max_accumulate) if val == True))[0] - 1
        space_left_open = np.abs(result[index_of_new_bin])
        result += space_left_open
        output += '|'
        output += (BIN_CAPACITY - space_left_open - 2) * 'X'
        output += '|'
        output += '_' * space_left_open
        output += '\n'
        if np.max(result) <= 0:
            break
        result -= BIN_CAPACITY
    return output

In [7]:
def tournamentSelector(population, reverse=False):
    random_indicies = np.random.randint(POPULATION_SIZE, size=TOURNAMENT_SIZE).tolist()
    tournament = []
    for idx, val in np.ndenumerate(random_indicies):
        tournament.append(population[val])
    results = []
    for val in tournament:
        terms = getMappedFitness(val)
        results.append(terms[0])

    results = np.array(results)
    if not reverse:
        pos = np.argmin(results)
    else:
        pos = np.argmax(results)
    return population[random_indicies[pos]], random_indicies[pos], results[pos]

In [8]:
def multipleSwapCrossover(p1, p2, swaps=4):
    draws = np.random.randint(PROBLEM_SIZE, size=swaps)

    c1 = p1.copy()
    c2 = p2.copy()

    for i, val in enumerate(draws):
        c1item = c1[val]
        c2item = c2[val]
        c1 = np.delete(c1, np.where(c1==c2item))
        c2 = np.delete(c2, np.where(c2==c1item))
        c1 = np.insert(c1, val, c2item)
        c2 = np.insert(c2, val, c1item)

    return c1, c2

In [9]:
def multipleMutator(p, swaps=4):
    draws = np.random.randint(PROBLEM_SIZE, size=(swaps, 2))

    child = p.copy()

    for i, val in enumerate(draws):
        tmp = child[val[0]]
        child = np.delete(child, val[0])
        child = np.insert(child, val[1], tmp)

    return child

In [10]:
def tryMutate(population):
    draw = np.random.rand()
    if draw < MUTATION_RATE:
        p, pos, fit = tournamentSelector(population)
        _, kpos, _ = tournamentSelector(population, reverse=True)

        c = multipleMutator(p, 1)

        population[kpos] = c
    return population


def tryCrossover(population):
    draw = np.random.rand()
    if draw < CROSSOVER_RATE:
        p1, p1pos, p1fit = tournamentSelector(population)
        p2, p2pos, p2fit = tournamentSelector(population)

        if any(p1 != p2):
            _, k1pos, _ = tournamentSelector(population, reverse=True)
            _, k2pos, _ = tournamentSelector(population, reverse=True)

            c1, c2 = multipleSwapCrossover(p1, p2, 3)

            population[k1pos] = c1
            population[k2pos] = c2
        else:
            p1 = multipleMutator(p1, swaps=int(PROBLEM_SIZE/5))

            population[p1pos] = p1

    return population

In [11]:
finalFitness = []
finalBins = []
bestFitnessOverTime = []
bestBinsOverTime = []

WITH_GRAPH_OUTPUT = False

for sample in range(SAMPLES):

    # Create new population
    population = []
    position = None
    chromosome = np.arange(PROBLEM_SIZE)
    for i in range(POPULATION_SIZE):
        np.random.shuffle(chromosome)
        population.append(chromosome.copy())
        
    bestBinsOverTime.append([])
    bestFitnessOverTime.append([])

    foundMin = False
    # Mutate and crossover for each generation
    for idx, generation in enumerate(range(GENERATIONS)):
        
        if foundMin == False:
            population = tryMutate(population)
            population = tryCrossover(population)

        if idx % SAMPLE_RATE == 0:
            print('GENERATION: ', idx)
            
            bins = []
            fitness = []
            for chromosome in population:
                terms = getMappedFitness(chromosome)
                bins.append(terms[0])
                fitness.append(np.array(terms[1]).mean())

            position = int(np.argmin(bins))
            print('Best in generation: ', bins[position], fitness[position])
            #print()
            #print(items[population[position]])

            bestFitnessOverTime[sample].append(fitness[position])
            bestBinsOverTime[sample].append(bins[position])
            
            if bins[position] == minBins:
                foundMin = True

    bins = []
    fitness = []
    for chromosome in population:
        terms = getMappedFitness(chromosome)
        bins.append(terms[0])
        fitness.append(np.array(terms[1]).mean())

    position = int(np.argmin(bins))
    print('Best in generation: ', bins[position], fitness[position])
    print()
    print(items[population[position]])

    finalFitness.append(fitness[position])
    finalBins.append(bins[position])

finalFitness = np.array(finalFitness)
finalBins = np.array(finalBins)

print('fitness: ')
print(finalFitness)
print('mean: ')
print(finalFitness.mean())
print('std deviation: ')
print('(' + str(finalFitness.std()) + ')')

print()
print(finalBins)
print('mean: ')
print(finalBins.mean())
print('std deviation: ')
print('(' + str(finalBins.std()) + ')')

GENERATION:  0
Best in generation:  29 16.06896551724138
GENERATION:  50
Best in generation:  28 13.071428571428571
GENERATION:  100
Best in generation:  28 13.071428571428571
GENERATION:  150
Best in generation:  28 13.071428571428571
GENERATION:  200
Best in generation:  28 13.071428571428571
GENERATION:  250
Best in generation:  28 13.071428571428571
GENERATION:  300
Best in generation:  28 13.071428571428571
GENERATION:  350
Best in generation:  28 13.071428571428571
GENERATION:  400
Best in generation:  28 13.071428571428571
GENERATION:  450
Best in generation:  28 13.071428571428571
Best in generation:  28 13.071428571428571

[99 67 27 99 72 22 51 21 24 40 46 96 23 33 30 14 91 42 11 44 17 61 85 69
 20 92 52 10 33 30 28  7 13  3 92 86 87 40 56 88 25 49 29 76 62 96 28 67
  7 74]
fitness: 
[13.07142857]
mean: 
13.071428571428571
std deviation: 
(0.0)

[28]
mean: 
28.0
std deviation: 
(0.0)


In [12]:
import matplotlib.pyplot as plt

iterations = []
knownBests = []
for i in range(GENERATIONS):
    if i % SAMPLE_RATE == 0:
        iterations.append(i)
        knownBests.append(minBins)


plt.close()

fig = plt.figure()
plt.grid(1)
plt.xlim([0, GENERATIONS])
plt.ion()
plt.xlabel('Generations')
plt.ylabel('Fitness')

plots = []
descriptions = []

bestFitnessOverTime_mean = np.mean(bestFitnessOverTime, axis=0)
bestBinsOverTime_mean = np.mean(bestBinsOverTime, axis=0)

plots.append(plt.plot(iterations, knownBests, 'ko', alpha=0.2, linewidth=0.5, markersize=3)[0])
plots.append(plt.plot(iterations, bestFitnessOverTime_mean, 'b--', linewidth=1, markersize=3)[0])
plots.append(plt.plot(iterations, bestBinsOverTime_mean, 'r-', linewidth=1, markersize=3)[0])
descriptions.append("Known best")
descriptions.append("Best fitness")
descriptions.append("Bins")

plt.legend(plots, descriptions)

plt.show()

<Figure size 640x480 with 1 Axes>